In [3]:
pip show purifytext

Name: purifytext
Version: 0.1.0
Summary: A package for cleaning and preprocessing text data
Home-page: 
Author: Aman Kumar Jha
Author-email: vats.amankumarjha2002@gmail.com
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: beautifulsoup4, contractions, nltk, numpy, pandas
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np 
import pandas as pd
from purifytext import clean_text
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.callbacks import EarlyStopping

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


2024-09-20 11:05:40.733093: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-20 11:05:40.733180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-20 11:05:40.735028: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
tf.__version__

'2.15.0'

In [6]:
df=pd.read_csv('IMDB Dataset\IMDB Dataset.csv')

In [7]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
df.shape

(50000, 2)

In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
df['sentiment'].value_counts().reset_index()

,sentiment,count
0,positive,25000
1,negative,25000


In [11]:
df_clean=clean_text(dataframe=df,column_name='review',stemming=True, lemmatizing=False)


=== Cleaning Process ===

⬇️ Removing HTML Tags ⬇️


/opt/conda/lib/python3.10/site-packages/purifytext/text_preprocessing.py:37: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")



⬇️ Lowercasing Text ⬇️

⬇️ Removing URLs ⬇️

⬇️ Removing Emojis ⬇️

⬇️ Removing Punctuation ⬇️

⬇️ Removing Special Characters ⬇️

⬇️ Removing Numbers ⬇️

⬇️ Removing Whitespace ⬇️

⬇️ Expanding Contractions ⬇️

⬇️ Removing Stopwords ⬇️

⬇️ Stemming Words ⬇️

=== Cleaning Completed ===



In [12]:
df_clean.head()

,review,sentiment
0,one review mention watch oz episod hook right ...,positive
1,wonder littl product film techniqu unassum old...,positive
2,thought wonder way spend time hot summer weeke...,positive
3,basic famili littl boy jake think zombi closet...,negative
4,petter mattei love time money visual stun film...,positive


In [13]:
df_clean['sentiment'] = df_clean['sentiment'].map({'positive': 1, 'negative': 0})

In [14]:
df_clean.head()

,review,sentiment
0,one review mention watch oz episod hook right ...,1
1,wonder littl product film techniqu unassum old...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic famili littl boy jake think zombi closet...,0
4,petter mattei love time money visual stun film...,1


In [15]:
X=df_clean['review'].values
y=df_clean['sentiment'].values

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
len(X_train),len(X_test),len(y_train),len(y_test)

(40000, 10000, 40000, 10000)

In [18]:
tokenizer=Tokenizer(oov_token="<nothing>")

In [19]:
tokenizer.fit_on_texts(X)

In [20]:
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 for padding
vocab_size

174564

In [21]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [22]:
max_len=500

In [23]:
X_train=pad_sequences(X_train,padding='pre',maxlen=max_len)
X_test=pad_sequences(X_test,padding='pre',maxlen=max_len)

In [24]:
X_train[0]

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [25]:
y_train[0]

0

In [26]:
X_train.shape

(40000, 500)

In [27]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [28]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len))
model.add(SimpleRNN(128, activation='relu')) 
model.add(Dense(1, activation='sigmoid'))

In [29]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          22344192  
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 22377217 (85.36 MB)
Trainable params: 22377217 (85.36 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, 
                    callbacks=[early_stopping_callback])

Epoch 1/10
1000/1000 [==============================] - 584s 582ms/step - loss: 0.6433 - accuracy: 0.7392 - val_loss: 0.3519 - val_accuracy: 0.8428
Epoch 2/10
1000/1000 [==============================] - 467s 467ms/step - loss: 0.7973 - accuracy: 0.8515 - val_loss: 0.3791 - val_accuracy: 0.8418
Epoch 3/10
1000/1000 [==============================] - 492s 492ms/step - loss: 0.1755 - accuracy: 0.9326 - val_loss: 0.3649 - val_accuracy: 0.8466
Epoch 4/10
1000/1000 [==============================] - 497s 497ms/step - loss: 0.1071 - accuracy: 0.9609 - val_loss: 0.4474 - val_accuracy: 0.8111
Epoch 5/10
1000/1000 [==============================] - 478s 478ms/step - loss: 0.1948 - accuracy: 0.9272 - val_loss: 0.5075 - val_accuracy: 0.7570
Epoch 6/10
1000/1000 [==============================] - 469s 469ms/step - loss: 0.2196 - accuracy: 0.9100 - val_loss: 0.5329 - val_accuracy: 0.8136


In [32]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

313/313 [==============================] - 14s 42ms/step - loss: 0.3419 - accuracy: 0.8488
Test Accuracy: 0.848800003528595
